# DeepLabV3+ Transfer Learning Training Demo

DeepLabV3+ w/ResNet101 backbone transfer learning training demo
Net pretrained with COCO val2017, then trained with Yamaha-CMU Off-Road Dataset

---

Authors:
- **Nate Haddad** haddad[dot]na[at]northeastern[dot]edu
- **Amit Mulay** mulay[dot]am[at]northeastern[dot]edu

In [ ]:
import torch

from torchvision import transforms, models
from torchvision.models.segmentation.deeplabv3 import DeepLabHead

import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt

from train import train_model
from dataset import get_dataloader, YamahaCMUDataset
from utils import display_example_pair, freeze_layers

In [ ]:
torch.cuda.is_available()

In [ ]:
np.random.seed(42)

Set up all of our path variables and hyperparamters needed to train the model. Adjust accordingly to your needs

In [ ]:
# image resize variables
IMG_HEIGHT = 513
IMG_WIDTH = 513

epochs = 5
num_mask_channels = 8
batch_size = 4

In [ ]:
DATA_PATH = 'data/yamaha_v0/'
MODEL_PATH = 'models/model_v0.1.{}.{}.pt'.format(batch_size, epochs)

Set up the `data` and `model` directories

In [ ]:
if not os.path.exists('data'):
    os.makedirs('data')
    
if not os.path.exists('models'):
    os.makedirs('models')

---

**IMPORTANT:** we require that you now download the Yamaha CMU Off-Road Dataset from the following url before proceding. The directory and files should be unzipped and placed in the `data` directory of this repository, created above.

**Link to download dataset:** https://theairlab.org/yamaha-offroad-dataset/

---

Load an example image from the training set

In [ ]:
example_image = Image.open('data/yamaha_v0/train/iid000008/rgb.jpg')
example_mask = Image.open('data/yamaha_v0/train/iid000008/labels.png')
image_display = np.array(example_image)
mask_display = np.array(example_mask.convert('RGB'))
display_example_pair(image_display, mask_display)

Load data into dataloaders

In [ ]:
dataloaders = get_dataloader(DATA_PATH, batch_size=batch_size, resize_shape=(IMG_HEIGHT,IMG_WIDTH))

### Model Selection

Create the model. Note that the model is pretrained on COCO val2017. Replace the output layer with 8 mask channels and the correct number of nodes for the last layer

ResNet50

In [ ]:
# model = models.segmentation.deeplabv3_resnet50(pretrained=True, progress=True, aux_loss=None)
# model.classifier = DeepLabHead(2048, num_mask_channels)

ResNet101

In [ ]:
# model = models.segmentation.deeplabv3_resnet101(pretrained=True, progress=True, aux_loss=None)
# model.classifier = DeepLabHead(2048, num_mask_channels)

MobileNetV3-Large

In [ ]:
model = models.segmentation.deeplabv3_mobilenet_v3_large(pretrained=True, progress=True, aux_loss=True)
model.classifier = DeepLabHead(960, num_mask_channels)

Freeze earlier layers

In [ ]:
# freeze_layers(model, 0, 14)

In [ ]:
model.train();

Select the optimizer and loss function

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

Train the model

In [ ]:
train_model(model, dataloaders, criterion, optimizer, num_epochs=epochs, is_inception=True);

---
Save the entire model

In [ ]:
torch.save(model, MODEL_PATH)